# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from pprint import pprint
import numpy as np

# Mapping Dependencies
import os
from bokeh.models import HoverTool
from holoviews import opts
import holoviews as hv
import warnings

warnings.filterwarnings("ignore")

# Import API key
from vacationpy_api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,newman,37.3138,-121.0208,287.28,88,100,3.58,US,1708212339
1,1,carnarvon,-24.8667,113.6333,303.01,34,1,5.39,AU,1708212340
2,2,lebu,-37.6167,-73.6500,288.06,77,0,11.17,CL,1708212341
3,3,puerto ayora,-0.7393,-90.3518,297.90,100,75,2.24,EC,1708212341
4,4,grytviken,-54.2811,-36.5092,278.72,78,81,1.32,GS,1708212342


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

width = 700
height = 500

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    title = "Humidity of Cities",
    tiles = "OSM",
    frame_width = width,
    frame_height = height,
    size = "Humidity",
    scale = 1,
    color = "Humidity",
).opts(width=width, height=height)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = city_data_df[
    (city_data_df['Humidity']<=35) &
    (city_data_df['Cloudiness']<=50) &
    (city_data_df['Wind Speed']<=3.00)
]

# # Drop any rows with null values
ideal_conditions = ideal_conditions.dropna()

# # Display sample data
ideal_conditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,fort st. john,56.2499,-120.8529,279.09,24,20,2.57,CA,1708212163
131,131,ghat,24.9647,10.1728,289.89,16,6,1.58,LY,1708212418
190,190,fort mcmurray,56.7268,-111.3810,276.88,32,0,1.03,CA,1708211935
199,199,boundiali,9.5217,-6.4869,299.20,13,10,0.51,CI,1708212460
292,292,as sulayyil,20.4607,45.5779,296.29,33,33,2.77,SA,1708212513
349,349,yambio,4.5721,28.3955,296.91,29,49,2.13,SS,1708212545
372,372,maridi,4.9158,29.4742,296.86,28,45,2.04,SS,1708212559
393,393,medina,24.4686,39.6142,289.39,20,0,2.06,SA,1708212571
429,429,korla,41.7597,86.1469,265.10,33,41,1.79,CN,1708212592
514,514,upington,-28.4478,21.2561,294.36,14,0,0.00,ZA,1708212660


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,fort st. john,CA,56.2499,-120.8529,24,
131,ghat,LY,24.9647,10.1728,16,
190,fort mcmurray,CA,56.7268,-111.3810,32,
199,boundiali,CI,9.5217,-6.4869,13,
292,as sulayyil,SA,20.4607,45.5779,33,
349,yambio,SS,4.5721,28.3955,29,
372,maridi,SS,4.9158,29.4742,28,
393,medina,SA,24.4686,39.6142,20,
429,korla,CN,41.7597,86.1469,33,
514,upington,ZA,-28.4478,21.2561,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : "accommodation.hotel",
    'apiKey' : {geoapify_key}
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'
    limit = 25
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)

    #f"https://api.geoapify.com/v2/places?categories={categories}&conditions={conditions}&filter={params['filter']}&limit={limit}&apiKey={geoapify_key}"

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
      
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
fort st. john - nearest hotel: Fort St. John Motor Inn
ghat - nearest hotel: فندق تاسيلي
fort mcmurray - nearest hotel: Nomad Hotel and Suites
boundiali - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
yambio - nearest hotel: Kabash Inn
maridi - nearest hotel: No hotel found
medina - nearest hotel: The Oberoi Madina
korla - nearest hotel: Silver Star Hotel
upington - nearest hotel: Die Eiland Holiday Ressort
atar - nearest hotel: فندق سكليل


,City,Country,Lat,Lng,Humidity,Hotel Name
95,fort st. john,CA,56.2499,-120.8529,24,Fort St. John Motor Inn
131,ghat,LY,24.9647,10.1728,16,فندق تاسيلي
190,fort mcmurray,CA,56.7268,-111.3810,32,Nomad Hotel and Suites
199,boundiali,CI,9.5217,-6.4869,13,No hotel found
292,as sulayyil,SA,20.4607,45.5779,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

width = 700
height = 500

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    title = "Hotels",
    tiles = "OSM",
    frame_width = width,
    frame_height = height,
    size = 15,
    scale = 2,
    color = ['Hotel Name'],
    hover_cols = ['City','Humidity','Hotel Name','Country'],
).opts(
    opts.Points(tools = ['hover'],width=width, height=height)
    
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)